In [2]:
import pandas as pd
import numpy as np
import json 

Note: when cleaning up, import data as modules 

### Loading Data and Preprocessing

Load data

In [33]:
with open('../acled_data/acleddata_jan_aug.json') as f:
    data = json.load(f)

Convert to pandas df

In [34]:
df = pd.DataFrame(data)

##### Converting fatalities to numeric format

In [35]:
df["fatalities"] = pd.to_numeric(df["fatalities"], errors='coerce')


In [36]:
df["latitude"] = pd.to_numeric(df["latitude"], errors='coerce')
df["longitude"] = pd.to_numeric(df["longitude"], errors='coerce')

#### Using spacy to extract data on wounded casualties from the 'notes' column and making a new feature - 'wounded' 

In [37]:
#Extracting the number of wounded from the notes column and adding a new column 'wounded'
import spacy
from word2number import w2n

nlp = spacy.load('en_core_web_md')

def extract_wounded(text):
    doc = nlp(text)
    wounded_count = 0

    for token in doc:
        if token.text.lower() == 'wounded':
            for child in token.children:
                if child.pos_ == 'NUM':
                    try:
                        wounded_count += int(child.text)
                    except ValueError:
                        wounded_count += w2n.word_to_num(child.text)
            if wounded_count == 0:
                for ancestor in token.ancestors:
                    if ancestor.pos_ == 'NUM':
                        try:
                            wounded_count += int(ancestor.text)
                        except ValueError:
                            wounded_count += w2n.word_to_num(ancestor.text)
                        break

    return wounded_count

df['wounded'] = df['notes'].apply(extract_wounded)


In [38]:
# Step 1a: Parse event_date into a datetime format and extract month and year
df["event_date"] = pd.to_datetime(df["event_date"])
df["month"] = df["event_date"].dt.month
df["year"] = df["event_date"].dt.year


In [39]:
#Combining fatalities + wounded into casualties
df['casualties'] = df['fatalities']+df['wounded']

In [40]:
# Step 1c: Aggregate data by location and month
# For example, we can sum fatalities and count the number of events for each location and month
grouped_columns = ["admin1", "admin2", "admin3", "location", "latitude", "longitude", "event_date", "year", "month"]
df_agg = df.groupby(grouped_columns).agg({"casualties": "sum", "event_type": "count"}).reset_index()
df_agg = df_agg.rename(columns={"event_type": "num_events"})


#### Normalizing the data with StandardScaler

In [41]:
from sklearn.preprocessing import StandardScaler

In [42]:
# Normalize the variables
scaler = StandardScaler()
normalized_df_agg = scaler.fit_transform(df_agg[['casualties', 'num_events']])

#### Dimensionality reduction with TSNE

In [43]:
from sklearn.manifold import TSNE

# Perform t-SNE
tsne = TSNE(n_components=1, random_state=42)
embedded_data = tsne.fit_transform(normalized_df_agg)

# Scale the t-SNE embeddings to a range of 0 to 100
scaled_embedded_data = (embedded_data - embedded_data.min()) / (embedded_data.max() - embedded_data.min()) * 100

df_agg['tsne_reduced'] = scaled_embedded_data


I thought it would be a good idea to create a feature that will represent the proximity of locations to known hot zones where armed clashes occur, which in turn increases the chances of artillery shelling, drones, and missiles strikes affecting said areas. For this batch, I will limit the pool of hot zones to 4 locations: Bakhmut, Soledar, Avdiivka, Vuhledar. Those locations are known as battlefronts where frequent heavy armed clashes between opposing forces occur therefore anything close to them is very likely to be affected. 

In order to create this feature, I will compute the distance between locations in the data set to the hot zones by calculationg the distance between two sets of latitude and longitude points by using the Haversine formula.

In [44]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Earth radius in kilometers
    earth_radius = 6371

    # Calculate the distance
    distance = earth_radius * c
    return distance


In [45]:
#Define coordinates of the the battlefront towns
bakhmut_coords = (48.5956, 37.9999)
soledar_coords = (48.6833, 38.0667)
avdiivka_coords = (48.1394, 37.7497)
vuhledar_coords = (48.7798, 37.2490)
robotyne_coords = (47.44992394238662, 35.83787190517212)
kupiansk_coords = (49.7160738622855, 37.596104878691285)

In [53]:
#apply the haversine function to the dataset to calculate 
#the distances to each town and find the minimum distance:
df_agg['distance_to_bakhmut'] = haversine(df_agg['latitude'], df_agg['longitude'], bakhmut_coords[0], bakhmut_coords[1])
df_agg['distance_to_soledar'] = haversine(df_agg['latitude'], df_agg['longitude'], soledar_coords[0], soledar_coords[1])
df_agg['distance_to_avdiivka'] = haversine(df_agg['latitude'], df_agg['longitude'], avdiivka_coords[0], avdiivka_coords[1])
df_agg['distance_to_vuhledar'] = haversine(df_agg['latitude'], df_agg['longitude'], vuhledar_coords[0], vuhledar_coords[1])
df_agg['distance_to_robotyne'] = haversine(df_agg['latitude'], df_agg['longitude'], robotyne_coords[0], robotyne_coords[1])
df_agg['distance_to_kupiansk'] = haversine(df_agg['latitude'], df_agg['longitude'], kupiansk_coords[0], kupiansk_coords[1])


df_agg['min_distance_to_battlefront'] = df_agg[['distance_to_bakhmut', 'distance_to_soledar', 'distance_to_avdiivka', 'distance_to_vuhledar', 'distance_to_robotyne', 'distance_to_kupiansk']].min(axis=1)

# Drop the temporary distance columns
df_agg = df_agg.drop(columns=['distance_to_bakhmut', 'distance_to_soledar', 'distance_to_avdiivka', 'distance_to_vuhledar', 'distance_to_robotyne', 'distance_to_kupiansk'])



#### Hazard Score 

Now that we have reduced the engineered features into 'tsne_reduced' to capture each locations intensity, we will now calculate a holistic hazard score based on each locations minimal distance to the battlefront. 

- First we will convert the 'min_distance_to_battlefront' feature into a distance score using exponential decay.
- Then we will combine the 'tsne_reduced' values (which already range between 0 to 100) and the 'distance_score' using a weighted sum to create the final hazard score:

In [70]:
# Convert distance to an exponential score
df_agg['exp_distance_score'] = np.exp(-alpha * df_agg['min_distance_to_battlefront'])

# Determine the weight for tsne_reduced
beta = 0.7

# Calculate hazard score
df_agg['hazard_score'] = beta * df_agg['tsne_reduced'] + (1 - beta) * df_agg['exp_distance_score'] * 100

# Optionally, scale the hazard score if necessary
# df_agg['hazard_score'] = (df_agg['hazard_score'] - df_agg['hazard_score'].min()) / (df_agg['hazard_score'].max() - df_agg['hazard_score'].min()) * 100


Due to unreported casualties, the current model suffers from memory loss. Hence, even if a location is close to the battlefield or is the battlefield, it might lose its hazard rating simply due to missing data in casualties. In order to fix this, we will apply a lingering effect, where a high casualty event will propagate its hazard score up to 2 weeks. The criteria for this effect is to have a min distance score of 80 (so that we ignore remote areas that suffer from a missile strike on civilians) and have a high casualty event occur within a week from the event in question.

In [80]:
# Initialize a new column to track when a high casualty event took place
df_agg['high_casualty_event_date'] = np.nan

# Populate the high_casualty_event_date column for rows where casualties > 0
df_agg.loc[df_agg['casualties'] > 0, 'high_casualty_event_date'] = df_agg['date']

# Forward fill the high_casualty_event_date column for 14 days or until another casualty event
df_agg['high_casualty_event_date'] = df_agg.groupby('location')['high_casualty_event_date'].transform(lambda x: x.ffill(limit=14))

# For rows where casualties = 0 and are within 14 days of a high casualty event, and distance_score is high,
# set the hazard score to be the same as the previous day
mask = (df_agg['casualties'] == 0) & (~df_agg['high_casualty_event_date'].isna()) & (df_agg['distance_score'] >= 80)
df_agg.loc[mask, 'hazard_score'] = df_agg.groupby('location')['hazard_score'].shift(1)

# Drop the high_casualty_event_date column, it's no longer needed
df_agg = df_agg.drop(columns=['high_casualty_event_date'])


Formatting data to handle inconsistencies with naming and missing values

In [48]:
df_agg.loc[df_agg['admin2']=='Kyiv', 'admin3'] = 'Kyiv'

In [49]:
df_agg.loc[df_agg['location']=='Kherson', 'admin2'] = 'Khersonskyi'

In [50]:
df_agg = df_agg[~df_agg['admin3'].str.strip().eq('')]


In [51]:
df_agg.to_csv('Hazards_latest.csv', index=False)

In [81]:
df_agg[df_agg['location']=='Robotyne']

,admin1,admin2,admin3,location,latitude,longitude,event_date,year,month,casualties,...,tsne_reduced,min_distance_to_battlefront,distance_score,hazard_score,inverse_distance,inverse_distance_score,log_distance_score,exp_distance_score,high_casualty_event,adjusted_hazard_score
25890,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-03-19,2023,3,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25891,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-03-22,2023,3,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25892,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-04-12,2023,4,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25893,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-04-18,2023,4,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25894,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-01,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25895,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-08,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25896,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-09,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25897,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-15,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25898,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-17,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
25899,Zaporizhia,Polohivskyi,Tokmatska,Robotyne,47.4478,35.837,2023-06-18,2023,6,0,...,0.000000,0.245103,0.997552,29.926559,4.079748,0.004079,1.406035,0.997552,1,29.926559
